# TP2: Ecrire comme votre auteur favori

Nous allons d'abord commencer par la problématique de prédiction de mots. L'idée est d'utiliser un livre pour apprendre à la machine dans quel ordre les mots ont le plus de chance d'apparaitre.

Dépot dans le cloud:
https://synapse.math.univ-toulouse.fr/s/2tqVI5vgZqsxaFX

## I. Téléchargement de livres depuis Project Gutenberg (https://www.gutenberg.org)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#List of books along with their identifier on Project Gutenberg
# Feel free to expand the list and pick your favorite
books = { "Alice's Adventures in Wonderland": 11  , # by Lewis Carroll, 160 kb
          "The Complete Poetical Works of Edgar Allan Poe": 10031, #by Edgar Allan Poe, 400 kb
          "The Hound of the Baskervilles":    2852, # by Sir Conan Doyle, 340 kb
          "The King James Bible":             10,   # by King James, 4.3 Mb
          "The Complete Works of William Shakespeare": 100 #by William Shakespeare, 5.5 Mb
        }

def bookSelection(bookTitle):
    bookId = books[bookTitle]
    #
    print( "Loading text from Project Gutenberg URL..." )
    import urllib3
    http = urllib3.PoolManager()
    # url = 'https://www.gutenberg.org/cache/epub/%s/pg%s.txt' % (bookId, bookId) # old url, website has changed
    url = 'https://www.gutenberg.org/files/%s/%s-0.txt' % (bookId, bookId)
    request = http.request('GET', url)
    #
    global book
    book = request.data
    print( "Request done with %d bytes..." % len(book))
    #Converts from bytes to string
    book = book.decode("utf-8") 
    return

interact(bookSelection, bookTitle=books.keys() );


interactive(children=(Dropdown(description='bookTitle', options=("Alice's Adventures in Wonderland", 'The Comp…

In [ ]:
import re #Rational expressions
words = re.split('[^A-Za-z]+', book.lower())
words = [w for w in words if w!=''] # Remove empty strings

# Computes a dictionary where keys are words
#  and values are the number of occurrences.
# Output is sorted
def generate1gram():
    gram = dict()
    
    # Populate 1-gram dictionary
    for i in range(len(words)-1):
        key = words[i]
        if gram.get(key) is None:
            gram[key] = 1
        else:
            gram[key] += 1
    
    #Warning! sorted forms a list
    gram = sorted(gram.items(), key=lambda x:-x[1])
    
    return dict(gram)

gram1 = generate1gram()

# Print length of list
print( "Total number of words, with repetition:    ", len(words) )
print( "Total number of words, without repetition: ", len(gram1) )

Total number of words, with repetition:     62955
Total number of words, without repetition:  5833


## II. Autocompletion avec des 1-grams

Exercice 1:
Ecrire une fonction findLastWord, dont l'entrée et la sortie sont des objets de type string. Etant donné une chaine de caractères, cette fonction doit isoler le dernier mot comme suit:
- transformer le tout en lettres minuscules
- enlever les espaces de fin
- trouver le dernier espace et isoler le dernier mot

In [ ]:
from IPython.display import display

# Information labels
infoLabel1 = widgets.Label(value="Input:")
infoLabel2 = widgets.Label(value="Last word:")
infoLabel3 = widgets.Label(value="Autocomplete suggestion:")
#Input area
inputArea = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='',
    disabled=False
)
# Output labels
outputLastWord = widgets.Label(value='')
outputLabel = widgets.Label(value='')
output = widgets.Output()

def findLastWord(str):
    str = str.lower()
    str = str.rstrip()
    # Find last word
    # Hint: Use https://docs.python.org/3.7/library/stdtypes.html?highlight=rfind#str.rfind
    lastWordIndex = str.rfind( ' ', 0, len(str))
    lastWord = ''
    if lastWordIndex == -1:
        return str
    else:
        return str[(lastWordIndex+1):]   

def onChange(change):
    with output:
        lastWord = findLastWord(change.new)
        outputLastWord.value = lastWord
        #
        #Either the last word exists in the dictionary and then nothing to do
        possibleWords = [ (w,x) for (w,x) in gram1.items() if w.startswith(lastWord)]
        outputLabel.value = ''.join( [ (w+'(%d) ')%x for (w,x) in possibleWords ] )

# Sets a callback upon the change of the TextArea's content
inputArea.observe(onChange, 'value')

display( infoLabel1, output)
display( inputArea, output)
display( widgets.HBox([infoLabel2, outputLastWord]), output)
display( infoLabel3, output)
display( outputLabel, output)

Label(value='Input:')

Output()

Textarea(value='', placeholder='Type something')

Output()

Output()

Label(value='Autocomplete suggestion:')

Output()

Label(value='')

Output()

In [ ]:
str='kabeu  % 6 ojhr 77 ihd;;;FYY---___gfuf       ufg  '.rstrip()+"ugu"


lastWordIndex = str.rfind( ' ', 0, len(str))
lastWordIndex
str[44+1:]

## III. Predire le mot suivant: 2-grams

Exercice 2:
 - Adapter la fonction précédente generate1gram et écrire la fonction generate2gram qui retourne un dictionnaire dont les clés sont les paires de mots et les valeurs le nombre d'occurences
 - Ecrire une fonction predictNextWord

In [ ]:
# Computes a dictionary where keys are all pairs of words
#  and values are the number of occurrences.
# Output is sorted
def generate2gram():
    gram = dict()
    
    # Populate 2gram dictionary
    for i in range(len(words)-1):
        key = ( words[i], words[i+1] )
        if gram.get(key) is None:
            gram[key] = 1
        else:
            gram[key] += 1
    
    #Warning sorted forms a list
    gram = sorted(gram.items(), key=lambda x:-x[1])
    
    return dict(gram)

In [ ]:
#Test
gram2 = generate2gram()
print( [(w,x) for (w,x) in gram2.items() if x>50] )

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
from IPython.display import display

# Information labels
infoLabel1 = widgets.Label(value="Input:")
infoLabel2 = widgets.Label(value="Autocomplete suggestions:")
infoLabel3 = widgets.Label(value="Suggestion from word prediction:")
#Input area
inputArea = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='',
    disabled=False
)
# Output labels
outputAutocomplete = widgets.Label(value='')
outputPrediction = widgets.Label(value='')
output = widgets.Output()

def onChange(change):
    with output:
        lastWord = findLastWord( change.new )
        #
        # Autocomplete as before
        possibleWords = [ (w,x) for (w,x) in gram1.items() if w.startswith(lastWord)]
        outputAutocomplete.value = ''.join( [ w+'(%d) '%x for (w,x) in possibleWords ] )
        #
        # Prediction
        # Either the last word does not exist in the dictionary (or equivalently the 1-gram) and then nothing to do
        if lastWord not in [w for (w,x) in possibleWords]:
            outputPrediction.value = ''
        # or the last word allows to predict the next
        else:
            possiblePairs = predictNextWord(lastWord)
            outputPrediction.value = ''.join( [ (p[1]+'(%d) ') % x for (p,x) in possiblePairs ] )
        
def predictNextWord(str):
    return [ i for i in gram2.items() if str==i[0][0]]

# Sets a callback upon the change of the TextArea's content
inputArea.observe(onChange, 'value')

display( widgets.HBox( [infoLabel1, inputArea] ) , output)
display( infoLabel2, output)
display( outputAutocomplete, output)
display( infoLabel3, output )
display( outputPrediction, output)


Output()

Label(value='Autocomplete suggestions:')

Output()

Label(value='')

Output()

Label(value='Suggestion from word prediction:')

Output()

Label(value='')

Output()

# III. N-Grams (tri-grams and more...)

Dans cette section, nous généralisons l'approche par chaines de Markov en prédisant le mot suivant à partir de plusieurs mots précédents. Pour cela, nous devons construire des Ngrams c'est-à-dire un dictionnaire de N-tuples de mots, en comptant les occurences.

Exercice 3:
1. Ecrire une fonction generateNgram
2. Comprendre les lignes de code suivantes
3. Remplir la fonction generate qui prend en entrée une chaine string et échantillonne un nouveau mot par prédiction de mot
4. Ecrire une boucle qui commence par une chaine vide et génère le mot suivant

In [ ]:
def generateNgram(n=1):
    gram = dict()
    
    # Some helpers to keep us from crashing the PC for now
    assert n > 0 and n < 100
    
    # Populate N-gram dictionary
    for i in range(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if gram.get(key) is None:
            gram[key] = 1
        else:
            gram[key] += 1
            
    #Warning sorted forms a list
    gram = sorted(gram.items(), key=lambda x:-x[1])
    return dict(gram)

In [ ]:
# Test
gram5 = generateNgram(3)
print( [(w,x) for (w,x) in gram5.items() if x>10] )

[]


In [ ]:
maxLag = 5
Ngrams = []
for i in range(1,maxLag):
    Ngrams.append( generateNgram(i))
Ngrams.reverse()
print( "Generated Ngrams for N from 1 to ", len(Ngrams) )

Generated Ngrams for N from 1 to  4


In [ ]:
def findLastWords(str, count=1):
    if count<= 0:
        return []
    str = str.lower()
    str = str.rstrip()
    #
    lastWordIndex = str.rfind( ' ', 0, len(str))
    lastWord = ''
    if lastWordIndex == -1:
        return [str]
    else:
        rec = findLastWords(str[:lastWordIndex], count-1)
        rec.append( str[(lastWordIndex+1):] )
        return rec

def predict(str):
    choices = []
    for gram in Ngrams:
        choice_i = [ (w,x) for (w,x) in gram.items() if w[:-1] == tuple( str[(len(str)-len(w)+1):] )]
        choices.append( choice_i )
    return choices

def predictMaxLag(list_of_words):
    for gram in Ngrams:
        choice = [ (w,x) for (w,x) in gram.items() if w[:-1] == tuple( list_of_words[(len(list_of_words)-len(w)+1):] )]
        if choice != []:
            return choice
    return []

import random
def weighted_choice(choice):
   total = sum(w for c, w in choice)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choice:
      if upto + w > r:
         return c
      upto += w
    
    
def generate(list_of_words):
    choice = predictMaxLag(list_of_words)
    value = weighted_choice(choice)
    return value[-1]

In [ ]:
strings = findLastWords( "king of the jungle and of the wild", count=3)
print( strings )
print("")
choice = predictMaxLag( strings )
print(choice)
gen = generate(strings)
print(gen)

In [ ]:
length=24
str = ''
for i in range(1,length):
    end = findLastWords(str, count=3)
    nextWord = generate(end)
    str = str + ' ' + nextWord
    print(str)